# **Data Collection**

## Objectives

* Fetch data from Kaggle and formulate for future operation.

## Inputs

* Kaggle JSON File which is an authentication token.

## Outputs

* The generated dataset will be inputs/cherryleaves_dataset/cherry-leaves.
- The dataset will conists of 3 folders presenting:
    - **train**, **test** & **validation**
- These folders will also consist of 2 further folders presenting:
    - **healthy** & **powdery_mildew**

## Additional Comments

* These steps are imperative to fetch the data, clean the data and then provide for the next step: Data Visualisation.



---

# Import Packages

In [1]:
! pip install -r /workspace/mildew-detection-in-cherry-leaves/requirements.txt

# Change Working Directory

Changing the working directory from its current folder to its parent folder
* Access the current directory with os.getcwd()

In [2]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detection-in-cherry-leaves/jupyter_notebooks'

Making the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
os.chdir('/workspace/mildew-detection-in-cherry-leaves')
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detection-in-cherry-leaves'

# Kaggle Install

In [5]:
! pip install kaggle

Changing Kaggle configuration directory to the current working directory and determine permissions for the Kaggle authentication JSON.

In [6]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Kaggle dataset and downloading.

In [7]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherryleaves_dataset"
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

Dataset URL: https://www.kaggle.com/datasets/codeinstitute/cherry-leaves
License(s): unknown
 95%|███████████████████████████████████▉  | 52.0M/55.0M [00:02<00:00, 19.5MB/s]
100%|██████████████████████████████████████| 55.0M/55.0M [00:02<00:00, 20.4MB/s]


Unzip the downloaded file and then delete the zip file.

In [8]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

---

# **Data Preparation**

## Data Clean

In [14]:
def removing_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        # printing(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location)
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file", len(j))
        print(f"Folder: {folder} - has non-image file", len(i))

In [15]:
removing_non_image_file(my_data_dir='inputs/cherryleaves_dataset/cherry-leaves')

IsADirectoryError: [Errno 21] Is a directory: 'inputs/cherryleaves_dataset/cherry-leaves/test/healthy'

## Divide Train, Test Validation

In [13]:
import os
import shutil
import random
! pip3 install joblib


def divide_train_validation_test_images(my_data_dir, train_set_ratio,
                                       validation_set_ratio, test_set_ratio):

    if train_set_ratio + validation_set_ratio + test_set_ratio != 1.0:
        print("train_set_ratio + validation_set_ratio"
              "+ test_set_ratio expected sum to 1.0")
        return

    # gets the classes labels
    labels = os.listdir(my_data_dir)  # expecting to get only the folder name
    if 'test' in labels:
        pass
    else:
        # creating train, test folders with the classes labels sub-folder
        for folder in ['train', 'validation', 'test']:
            for label in labels:
                os.makedirs(name=my_data_dir + '/' + folder + '/' + label)

        for label in labels:

            files = os.listdir(my_data_dir + '/' + label)
            random.shuffle(files)

            train_set_files_qty = int(len(files) * train_set_ratio)
            validation_set_files_qty = int(len(files) * validation_set_ratio)

            count = 1
            for file_name in files:
                if count <= train_set_files_qty:
                    # moving a secleted file to the train set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/train/'
                                + label + '/' + file_name)

                elif count <= (train_set_files_qty + validation_set_files_qty):
                    # moving a secleted file to the validation set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/validation/' + label + '/'
                                + file_name)

                else:
                    # moving a secleted file to test set
                    shutil.move(my_data_dir + '/' + label + '/' + file_name,
                                my_data_dir + '/test/' + label + '/'
                                + file_name)

                count += 1

            os.rmdir(my_data_dir + '/' + label)

- Training set is divided into a **0.70** ratio of data.
- Test set is divided into a **0.20** ratio of data.
- Validation set is divided into a **0.10** ratio of data.

In [13]:
divide_train_validation_test_images(my_data_dir="inputs/cherryleaves_dataset/cherry-leaves",
                                   train_set_ratio=0.7,
                                   validation_set_ratio=0.1,
                                   test_set_ratio=0.2
                                   )